In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim

In [2]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.4), # 随机水平翻转
    transforms.RandomVerticalFlip(0.4), # 随机垂直翻转
    transforms.ColorJitter(0.5,0.5,0.5,0.5), # 亮度、对比度、饱和度、色彩度
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

In [3]:
# 训练集
trainset = datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
# 测试集
testset = datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 批量读取数据
BATCH_SIZE = 32
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)

In [8]:
# 定义网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()     
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3,padding=1) 
        self.conv3 = nn.Conv2d(32, 32, 5)
        self.pooling = nn.MaxPool2d(2)  
        self.fc1 = nn.Linear(32*2*2, 64)
        self.fc2 = nn.Linear(64, 32)    
        self.fc3 = nn.Linear(32, 10) 
        
    def forward(self, x):
        batch_size=x.size(0)
        x=F.relu(self.pooling(self.conv1(x)))
        x=F.relu(self.pooling(self.conv2(x)))
        x=F.relu(self.pooling(self.conv3(x)))
        x=x.view(batch_size,-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model=Net()

In [9]:
criterion = nn.CrossEntropyLoss() # 交叉式损失函数
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9) # 优化器5

In [10]:
# 定义轮数
EPOCHS = 150

for epoch in range(EPOCHS):
    if (epoch+1) % 10 == 0:  # 每迭代次，更新一次学习率        
        for params in optimizer.param_groups:
            if params['lr'] > 0.001:
                params['lr'] *= 0.8
            else:
                params['lr'] *= 0.9
            print("lr : %.7f"%(params['lr']))
            
    train_loss = 0.0
    for i, (datas, labels) in enumerate(train_loader):
        # 梯度置零
        optimizer.zero_grad()
        # 训练
        outputs = model(datas)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
        # 累计损失
        train_loss += loss.item()
    # 打印信息
    print("Epoch :%d , Batch : %5d , Loss : %.4f"%(epoch+1, i+1, train_loss/len(train_loader.dataset)))

Epoch :1 , Batch :  1563 , Loss : 0.0720
Epoch :2 , Batch :  1563 , Loss : 0.0711
Epoch :3 , Batch :  1563 , Loss : 0.0653
Epoch :4 , Batch :  1563 , Loss : 0.0582
Epoch :5 , Batch :  1563 , Loss : 0.0541
Epoch :6 , Batch :  1563 , Loss : 0.0503
Epoch :7 , Batch :  1563 , Loss : 0.0472
Epoch :8 , Batch :  1563 , Loss : 0.0446
Epoch :9 , Batch :  1563 , Loss : 0.0429
lr : 0.0016000
Epoch :10 , Batch :  1563 , Loss : 0.0406
Epoch :11 , Batch :  1563 , Loss : 0.0393
Epoch :12 , Batch :  1563 , Loss : 0.0382
Epoch :13 , Batch :  1563 , Loss : 0.0372
Epoch :14 , Batch :  1563 , Loss : 0.0366
Epoch :15 , Batch :  1563 , Loss : 0.0358
Epoch :16 , Batch :  1563 , Loss : 0.0351
Epoch :17 , Batch :  1563 , Loss : 0.0345
Epoch :18 , Batch :  1563 , Loss : 0.0340
Epoch :19 , Batch :  1563 , Loss : 0.0336
lr : 0.0012800
Epoch :20 , Batch :  1563 , Loss : 0.0329
Epoch :21 , Batch :  1563 , Loss : 0.0323
Epoch :22 , Batch :  1563 , Loss : 0.0319
Epoch :23 , Batch :  1563 , Loss : 0.0316
Epoch :24 , B

In [21]:
# 保存模型
PATH = './cifar_model.pth'

torch.save(model.state_dict(), PATH)

In [8]:
# 加载模型

model = Net()
path = './cifar_model.pth'
model.load_state_dict(torch.load(path)) 

<All keys matched successfully>

In [20]:
# 测试
correct = 0
total = 0
with torch.no_grad():
    for i , (datas, labels) in enumerate(test_loader):
        # 输出
        outputs = model(datas) # outputs.data.shape --> torch.Size([128, 10])
        _, predicted = torch.max(outputs.data, dim=1) # 第一个是值的张量，第二个是序号的张量
        # 累计数据量
        total += labels.size(0)  # labels.size() --> torch.Size([128]) , labels.size(0) --> 128
        # 比较有多少个预测正确
        correct += (predicted == labels).sum() # 相同为1，不同为0，利用sum()求总和
    print("在10000张测试集图片上的准确率：{:.3f}%".format(correct / total * 100))

在10000张测试集图片上的准确率：70.280%
